# Custom Tools Demo
<br/>
<img src="https://eu-images.contentstack.com/v3/assets/blt6b0f74e5591baa03/blt5d5323f706ef288f/637c0195c162df49beaae102/Untitled_design_(77).png?width=1280&auto=webp&quality=95&format=jpg&disable=upscale" width=500 /><br/>

## Part 1: Building & Prototyping Agent Tools
In this notebook we'll examine how we can build agents from simple API calls. Since we're not hard coding the API POST payload we need to few-shot prompt how we're going to do this with an example of the structure the API in question is expecting. This will allow us to build up a collection of tools. For the sake of demonstration we'll be separating out 3 tools across two different agents. 

In [0]:
%pip install --upgrade --quiet langchain-core langchain-databricks langchain databricks-vectorsearch langchain-community feedparser
%pip install transformers sentence_transformers
%pip install mlflow

dbutils.library.restartPython()

**LLMs (LLM + Prompt)**

- A standalone LLM responds to text prompts based on knowledge from a vast training dataset.
- Good for simple or generic queries but often disconnected from your real-world business data.
<img src="https://learn.microsoft.com/en-us/azure/databricks/_static/images/generative-ai/llms.svg" />

**Hard-coded agent system (“Chain”)**

- Developers orchestrate deterministic, pre-defined steps. For example, a RAG application can always retrieve from a vector store and combined results with the user prompt.
- The logic is fixed, and the LLM does not decide which tool to call next.
<img src="https://learn.microsoft.com/en-us/azure/databricks/_static/images/generative-ai/llm-tools.svg" />

**Tool-calling agent system**

- The LLM decides which tool to use and when to use it at runtime.
- This approach supports dynamic, context-aware decisions about which tools to invoke, such as a CRM database or a Slack posting API.
<img src="https://learn.microsoft.com/en-us/azure/databricks/_static/images/generative-ai/ai-agent.svg" />

**Multi-agent systems**

- Multiple specialized agents, each with its own function or domain.
- A coordinator (sometimes an AI supervisor, sometimes rule-based) decides which agent to invoke at each step.
- Agents can hand tasks off to each other while preserving the overall conversation flow.
<img src="https://learn.microsoft.com/en-us/azure/databricks/_static/images/generative-ai/multi-agent-system.svg" />

## Simulating a few API calls
We're going to build some tools around some API calls. For that we'll need to find a few good analogs for things like time series data
<br/>
<br/>
**Simulate time series retrieval**<br/>
	•	REST API for weather forecasts and historical data.<br/>
	•	Simulates querying sensor data or time series like in Cognite.<br/>
GET https://api.open-meteo.com/v1/forecast?latitude=51.0478&longitude=114.0593&hourly=temperature_2m<br/>
<br/>
**Simulate asset metadata catalog** <br/>
This is a good example of proprietary asset storage. This could be coming either from an external API or a delta table. For the latter, I'd opt for using a Genie agent or an online table feature spec instead, but we can do it either way.<br/>
	•	Can simulate fetching metadata about real-world locations (assets).<br/>
	•	Think of this as querying a catalog of objects.<br/>
GET https://nominatim.openstreetmap.org/search?q=Statue+of+Liberty&format=json<br/>
<br/>
**Simulate a document search**<br/>
General search and lookup of online documentation. <br/>
	•	Useful for document-based retrieval like CDF events or files.<br/>
	•	Great for agent retrieval demos with RAG.<br/>
GET http://export.arxiv.org/api/query?search_query=all:ai&start=0&max_results=2<br/>

## Getting started with a prototype
The fastest way to get started is to do a simple request call to each of the endpoints. If we put each of our APIs in their own function we can do rapid development with some declarative tools.

In [0]:
#Let's create some sample functions to test our endpoints. We can call these just to get a sense of how they behave and what kind of input parameters we'll need to inject.

import requests

def fetch_weather(latitude: float, longitude: float):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
    }
    response = requests.get(url, params=params)
    return response.json()

def search_assets(query: str):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": query, "format": "json"}
    response = requests.get(url, params=params)
    return response.json()

def search_arxiv(keywords: str, max_results: int = 3):
    url = "http://export.arxiv.org/api/query"
    params = {
        "search_query": f"all:{keywords}",
        "start": 0,
        "max_results": max_results
    }
    response = requests.get(url, params=params)
    return response.text  # XML format, you can parse with `feedparser`

In [0]:
#Let's do a quick preview of our data. The easiest way to do this is to throw it into a dataframe. We're just looking for a successful call and loose schema.
# We can do the same for any of our API calls

import pandas as pd

#Let's get a sample from Calgary
pd.DataFrame(fetch_weather(51.0478, 114.0593))

### Structured data
Obtaining structured data from an API makes the query results easier for the LLM to comprehend. Since there's little in terms of context required, LLMs tend to perform better and hallucinate less with structured or semi-structured data.

## (Option 1): Using LangChain's tools library to create tool stubs
This is a pretty easy approach, using a declarative approach and LangChain's tool library. All you need to do is register a function with a name and detailed description that will help the agent understand what each tool does.

This approach comes with a tradeoff; we lose flexibility with how our tools behave with the APIs. Since we'll want to tailor this behavior when we POST the payload, and retrieve only certain attributes this method is really only suitable for prototyping and rapid development.

In [0]:
#LangChain provides a quick and easy way to log functions as tools. This is an 'easy button' but we lose a degree of control in terms of behaviour management. Nonetheless, it's a pretty easy way to get going if you just want to do something simple.

#Although we probably won't be using this set of tools, it's good to know that it's an option for rapid development or prototyping.

from langchain.tools import Tool

tool_box = [
    Tool.from_function(fetch_weather, name="FetchWeather", description="Fetch hourly weather data."),
    Tool.from_function(search_assets, name="SearchAssets", description="Search for asset-like locations."),
    Tool.from_function(search_arxiv, name="SearchArxiv", description="Search research documents."),
]

## (Option 2): Creating our own tools using LangChain's Abstract BaseTool() class
There are some benefits to building our own tools based on abstract declarations from LangChain. Custom tools have added support for extended behaviours that simply aren't available when we register a tool using the from_function() .... function. This allows us to tailor our tools better to our needs, and we can build more complex functions that make external calls and have their own dependencies.

### Tool chains and tool boxes
**Toolboxes**
- A toolbox is a collection of discrete tools or functions that an agent can call upon to complete tasks. We will be using these for our agents. 
- Toolboxes contain a collection of atomic functions that serve a common purpose. Generally speaking, an agent owns a single toolbox.

- Tools are atomic: each tool does one thing well (e.g., run SQL, fetch from API, classify text).
  - Tools are often defined with a schema: inputs/outputs are validated, often with decorators or Pydantic models.
  - Tools may be:
  - Internal (Python functions, DB queries)
  - External (REST API calls, MLflow endpoints, vector search)
  - Stateful (accessing memory, logs, or intermediate context)
  - Agents choose tools based on prompts, planning modules, or learned decision policies.

**Tool Chains**
- A tool chain is an orchestrated sequence (or graph) of tools, where the output of one tool becomes the input for the next.
- Tool chains are often implemented as a workflow or graph in compound AI systems.
  - Sequential or branching logic (e.g., if → then → else).
  - Deterministic or probabilistic planning (static routing vs planner-based).
  - Can incorporate LLM-based planners or rule-based logic.
  - Often built with LangGraph, LangChain chains, DSPy programs, or even custom DAGs.

### Agent expectations
Since we're going to be leveraging LangChain's BaseTool and BaseModel classes, we need to implement them properly. `BaseModel()` will provide us with an input interface and `BaseTool()` will provide us with the structured framework we will rely on for our application. It's important to choose an agent framework that's consistent with how we're going to make use of our agents later on. When we create an implementation of the `BaseTool()` class, we're obligated to include 2 attributes (`name` and `description`) and two behaviors (`_run` and `_arun`).

In [0]:
#Note: normally we wouldn't worry about re-declaring our library imports, but in this case we are so the cells can be copied and pasted elsewhere to other notebooks.

from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field
import requests

#Input Schema for the weather class. We'll be using this to compose the tool
class WeatherInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

#Create an implementation inheriting the BaseTool abstract class from LangChain. _run() and _arun() are both required implementations. The name and description attributes are also required.
class FetchWeatherTool(BaseTool):
    name: str = "fetch_weather"
    description: str = "Fetch hourly weather temperature data for a given latitude and longitude. When asked about weather, assume that the user is always referring to temperature only. temperature_2m refers to the temperature in degrees celsius."
    args_schema: Type[BaseModel] = WeatherInput #This is the input schema we defined above.

    #The _run() function is always called when invoked. It's essentially doing the same thing as a class constructor, but since we're invoking the class from a toolchain in lieu of instancing the class as an object, this is run instead. This behaviour is what we're inheriting from the BaseTool() class.
    def _run(self, latitude: float, longitude: float):
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "hourly": "temperature_2m",
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()["hourly"]["temperature_2m"][:5]  # Preview
        return f"Failed to fetch data: {response.status_code}"

    def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async not supported.")

### Repeating the process for the remaining tool defintions
Next, we'll create a few more classes for the remaining tools. The structure is pretty much the same as the last agent we defined. We'll do one for the asset search and another for a document search.

In [0]:
from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field
import requests
import feedparser


class ArxivSearchInput(BaseModel):
    query: str = Field(..., description="Search keywords for the arXiv research papers")
    max_results: Optional[int] = Field(5, description="Max number of papers to return")

class SearchArxivTool(BaseTool):
    name: str = "search_arxiv"
    description: str = "Search for academic papers on arXiv related to a given keyword."
    args_schema: Type[BaseModel] = ArxivSearchInput

    def _run(self, query: str, max_results: int = 3):
        url = "http://export.arxiv.org/api/query"
        params = {
            "search_query": f"all:{query}",
            "start": 0,
            "max_results": max_results,
        }
        response = requests.get(url, params=params)
        feed = feedparser.parse(response.text)
        results = []
        for entry in feed.entries[:max_results]:
            results.append(f"{entry.title} — {entry.link}")
        return "\n".join(results) if results else "No papers found."

    def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async not supported.")


In [0]:
from langchain.tools import BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field
import requests

class AssetSearchInput(BaseModel):
    asset_query: str = Field(..., description="The name of the location or asset to search for")

class SearchAssetsTool(BaseTool):
    name: str = "search_assets"
    description: str = "Search for geographic or structural asset metadata using OpenStreetMap. "
    args_schema: Type[BaseModel] = AssetSearchInput

    def _run(self, asset_query: str):
        url = "https://nominatim.openstreetmap.org/search"
        params = {"q": asset_query, "format": "json"}
        headers = {"User-Agent": "LangChainAgent/1.0 (andrij.demianczuk@databricks.com)"}

        response = requests.get(url, params=params, headers=headers)
        if response.status_code != 200:
            return f"Search API returned {response.status_code}. Cannot continue using search_assets."
        
        data = response.json()
        if data:
            result = data[0]
            return f"{result['display_name']} (lat: {result['lat']}, lon: {result['lon']})"
        else:
            return f"No results found for '{asset_query}'"

    def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async not supported.")

## Building an agent that will use the tools.
Now that we have defined our tools, we must utilize them. We're going to need a few things to tie it all together including an instance of our LLM, an initializer and an agent description.

Let's get started and make use of one of the UC-provided foundation models. For it's ease of implementation, we'll use Anthropic's Claude 3.7 Sonnet for this project.

In [0]:
from langchain_databricks import ChatDatabricks

#This is the foundation LLM that we'll be using for the basis of our agents
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, extra_params={"temperature": 0.3})

### Testing the instance of the LLM
Let's just run a quick test to make sure our foundation model endpoint is available and providing results. This is going to serve as the `router` for our tools and for our agents.

In [0]:
print(llm.invoke('What are Calgarys coordinates?'))

## Agent Simulation 1: Weather & Research Articles
The first agent we'll build will contain two tools, stored in a dynamic array (list). This collection of tools is referred to as a 'toolbox'. This prescribes what the agent has to work with. In our case, since we defined these tools as API calls, the agent is responsible for translating the conversation into something the APIs can understand. When we initialize our agent, it's also important to declare an agent type. This tells LangChain how the agent is expected to behave and requires certain parameters to work properly. How we declared our tools also has an impact on the agent type we choose. Since we're using a structured, memory-enabled format, we'll opt for a structured chat with zero-shot ReAct (reason-action) type description.

A full explanation and list of LangChain agent types [can be found here](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html)

In [0]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

import os
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

#Define the tools (toolbox) available to our agent
tools = [
    FetchWeatherTool(),
    SearchArxivTool(),
]

# Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=10,
    return_messages=True
)

aibot = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=10
)

### Prompt engineering with tools
Since our input prompts for the agents require a specific format based on the tools (remember, each API has it's own signature when called), we'll need to create a prompt based on the tooling interfaces. Using `llm_chain` tells our system that it has access to any of the tools in the toolbox. If we wanted to prescribe how the chaining of tools works, we'd create a static chain instead.

In [0]:
new_prompt = aibot.agent.create_prompt(
    tools=tools
)

aibot.agent.llm_chain.prompt = new_prompt

## Agent Simulation 2: Custom Asset Lookups
The second agent definition is much the same as the first, just with a differen toolbox available to it.

In [0]:
#The second agent will be responsible for doing asset lookups. If we had a known set of our assets that our foundational model isn't aware of (e.g., OT asset locations in the field), we could use it to get our coordinates. When we create our composition of agents we'll declare descriptions for each agent.

from langchain.agents import initialize_agent
from langchain.agents import AgentType

import os
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

#We're going to use this to create a second agent for a multi-agent approach
asset_tools = [
    SearchAssetsTool()
]

# Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=10,
    return_messages=True
)

asset_bot = initialize_agent(
    tools=asset_tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=10
)

In [0]:
asset_prompt = asset_bot.agent.create_prompt(
    tools=asset_tools
)

asset_bot.agent.llm_chain.prompt = asset_prompt

## Testing our tools with both agents
Now that we've created our two agent prototypes, we can run them through a litany of tests.

### Warts and all.....
Now, at the time of this writing there are still some bugs and limitations present in these agents and their logic. This is important to acknowledge as it shows development and growth both within the applicaiton and on a personal level. I believe it's essential to document these blemishes as they progress. Understanding systems is just as critical as being able to code them.

- There is a bug when asking the asset agent for items not in its inventory. It loops until it times out without respecting the exit clause
- The internal conversations are overly apologetic. This either has something to do with the llm temperature or the way I wrote the template. Will fix later.
- Using defined agent types is going to be deprecated. [LangChain has recommended the updated use of LangGraph instead](https://python.langchain.com/docs/how_to/migrate_agent/). **This will be updated in the next release (as of 20/May/2025 - pinky promise!!!)**

In [0]:
aibot_output = aibot("Can you find me some article references for AI?")

In [0]:
print(aibot_output['output'])

In [0]:
aibot_output = aibot("Can you find me the address for the White House in Washington DC?")

In [0]:
print(aibot_output['output'])

In [0]:
aibot_output = aibot("What is the temperature at 51.0447° N, 114.0719° W?")

In [0]:
print(aibot_output['output'])

In [0]:
aibot_output = aibot("What are the lat and lon coordinates of West Edmonton Mall?")

In [0]:
print(aibot_output['output'])

In [0]:
aibot_output = aibot("What is the weather around West Edmonton Mall?")

In [0]:
print(aibot_output['output'])

In [0]:
asset_output = asset_bot("What are the coordinates of the Saddledome in Calgary AB?")

## Assembling our new agents into an compound AI system
Now that we have our two agents (and by extension with two toolboxes), we can start putting together our agentic application. We'll use the Databricks Agent framework to help us out with this. We'll need to coalesce and decouple our agents (including a supervisor) into new files and publish them to MLFlow for serving. Since this is a big task, we'll take care of that in the next notebook `02-Multi-Agent_Resoning_Demo_with_Custom_Tools`.

### To be continued...
In the next notebook `02-Multi-Agent_Resoning_Demo_with_Custom_Tools` we will take the logic and prototypes we built here and start composing a larger application that has access to both of our agents and by extension all of our tools for single publication. This is where we'll really getting into a true *compound* AI system - Where AI agents can talk to one another directly.